## Entrainement du modèle

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import joblib

# Charger les données
df_vecteur = pd.read_csv('../../data/processed/clean_dataset.csv')

# Séparer les caractéristiques et la cible
X = df_vecteur['merged']
y = df_vecteur["prdtypecode"]

# Encoder la cible 
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Séparer les données
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Définir le pipeline pour l'entrainement
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('smote', SMOTE(random_state=42)),
    ('svc', SVC(kernel='linear', probability=True, random_state=42))
])

# Entraîner le pipeline
pipeline.fit(X_train, y_train)

# Prédictions
y_pred = pipeline.predict(X_test)

# Évaluation
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_.astype(str)))

# Extraire les objets entraînés (sans SMOTE)
tfidf_trained = pipeline.named_steps['tfidf']
svc_trained = pipeline.named_steps['svc']

# Définir le pipeline pour l'inférence (ie sans SMOTE) et avec sklearn.pipeline pour la compatibilité avec le VotingClassifier
from sklearn.pipeline import Pipeline as SklearnPipeline

inference_pipeline = SklearnPipeline([
    ('tfidf', tfidf_trained),
    ('svc', svc_trained)
])
 
# Sauvegarde du pipeline complet
joblib.dump(inference_pipeline, 'inference_pipeline_svc_model.joblib')

# Sauvegarde du label encoder
joblib.dump(label_encoder, 'label_encoder.joblib')



In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import joblib

# Charger les données
df_vecteur = pd.read_csv('../../data/processed/clean_dataset.csv')

# Séparer les caractéristiques et la cible
X = df_vecteur['merged']
y = df_vecteur["prdtypecode"]

# Encoder la cible 
label_encoder = joblib.load('label_encoder.joblib')
y_encoded = label_encoder.transform(y)

# Séparer les données
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

svm_pipeline = joblib.load('inference_pipeline_svc_model.joblib')

# Prédictions
y_pred = svm_pipeline.predict(X_test)

# Évaluation
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_.astype(str)))

              precision    recall  f1-score   support

          10       0.21      0.58      0.30       397
          40       0.49      0.43      0.46       295
          50       0.62      0.72      0.67       225
          60       0.82      0.90      0.86       103
        1140       0.58      0.59      0.59       309
        1160       0.79      0.52      0.62       487
        1180       0.20      0.26      0.23       108
        1280       0.64      0.45      0.53       669
        1281       0.44      0.46      0.45       282
        1300       0.80      0.79      0.79       670
        1301       0.52      0.61      0.56        79
        1302       0.64      0.67      0.65       299
        1320       0.64      0.57      0.60       391
        1560       0.74      0.72      0.73       641
        1920       0.83      0.88      0.85       528
        1940       0.49      0.62      0.55       122
        2060       0.76      0.71      0.73       681
        2220       0.66    